# Computational Biology Seminar
## Project 2: Genomics

The goal for this project is to do a genome-wide association study (GWAS) using a mock dataset (since the real datasets required for this typically aren't publicly available).

In [1]:
import csv
import numpy as np
from scipy.stats import chi2_contingency

### **Step 1:** Reading in the data

In our mock dataset, there is a row for each SNP. For each row, the first three columns contain the information about the SNP: snp_id (chromosome number and position), the reference nucleotide, and the variant nucleotide. 

Then, there are 200 columns, corresponding to the 200 samples labeled in the heading row. The first 100 are the control samples (those that didn't turn into zombies). The next 100 are the "case" samples -- those that did turn into zombies. For each SNP, the column for each samples contains a 0, 1, or 2. This tells us whether that person was homozygous for the reference nucleotide (0), heterozygous (1), or homozygous for the variant nucleotide.

To start, let's read in all this data. We'll store the first three columns in a list called `snp_info`, and the rest in a list of lists called `data`.

In [2]:
snp_info = []
data = []
with open("data/example_data.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')

    for row in reader:

        snp_info.append(row[0:3])
        data.append(row[3:])

The first row was the column headings, so we'll remove that from both `snp_info` and `data` and store it separately.

In [3]:
snp_header = snp_info[0]
snp_info = snp_info[1:]

samples = data[0]
data = data[1:]

Next, we'll cast all these lists to NumPy arrays to make them easier to work with. And remember, we need to cast the elements in `data` to floats (or ints in this case if we wanted) since they were read in as strings by default.

In [4]:
snp_info = np.array(snp_info)
data = np.array(data).astype(float)

snp_header = np.array(snp_header)
samples = np.array(samples)

Quick shape check...

In [5]:
print(snp_info.shape)
print(data.shape)
print()
print(snp_header.shape)
print(samples.shape)

(48752, 3)
(48752, 200)

(3,)
(200,)


Quick content check...

In [6]:
print("snp_header:")
print(snp_header)
print()

print("samples:")
print(samples[0:4])
print()

print("snp_info:")
print(snp_info[0:5])
print()

print("data:")
print(data[0:5,0:4])
print()

snp_header:
['snp_id' 'ref' 'var']

samples:
['normal_1' 'normal_2' 'normal_3' 'normal_4']

snp_info:
[['chr1:944564' 'T' 'C']
 ['chr1:950243' 'A' 'C']
 ['chr1:959842' 'C' 'T']
 ['chr1:990417' 'T' 'C']
 ['chr1:1297065' 'C' 'T']]

data:
[[0. 1. 1. 1.]
 [2. 2. 1. 1.]
 [1. 2. 1. 2.]
 [0. 2. 2. 2.]
 [2. 1. 0. 1.]]



Next we want to make an array with the case/control (zombie/normal) labels for each sample. We can do this with some pretty basic string manipulation.

In [7]:
labels = []

for i in range(len(samples)):
    tmp = samples[i].split("_")
    labels.append(tmp[0])
    
labels = np.array(labels)

Quick checks...

In [8]:
print(labels[0:5])

print(set(labels))

['normal' 'normal' 'normal' 'normal' 'normal']
{'zombie', 'normal'}


### **Step 2:** GWAS analysis

Now, check each SNP to see if its distribution differs significantly between the zombie/normal phenotypes. To do this, we'll use the [chi-squared test.](https://en.wikipedia.org/wiki/Chi-squared_test) If you're not familiar with this test, I recommend checking out these Khan Academy videos to learn about it: [video 1,](https://www.youtube.com/watch?v=dXB3cUGnaxQ) [video 2,](https://www.youtube.com/watch?v=2QeDRsxSF9M) and [video 3.](https://www.youtube.com/watch?v=hpWdDmgsIRE) To do this, we'll use the `chi2_contingency` function that we imported from `scipy.stats` at the top.

In [39]:
from scipy.stats import chi2_contingency
y = np.where(labels == "zombie", 1, 0)

i = 4523

snp_genotypes = data[i, :]

contingency_table = np.zeros((2, 3))
for j in range(len(snp_genotypes)):
    genotype = int(snp_genotypes[j])
    status = y[j]
    contingency_table[status, genotype] += 1


chi2, p_value, _, _ = chi2_contingency(contingency_table)


print(p_value)


0.5515836561577222
